# Requirements

In [1]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install peft

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 16.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13

# Import data from HuggingFaceHub

In [2]:
from datasets import load_dataset

In [31]:
trainset= load_dataset("allocine", split="train").remove_columns("label")
testset= load_dataset("allocine", split="test").remove_columns("label")

In [4]:
trainset

Dataset({
    features: ['review'],
    num_rows: 160000
})

# Model tokenizer

In [5]:
from transformers import AutoTokenizer

In [26]:
tokenizer_name_or_path = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/config.json
Model config GPTNeoConfig {
  "_name_or_path": "EleutherAI/gpt-neo-125M",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_types": [
    [
      [
        "global",
        "local"
      ],
      6
    ]
  ],
  "bos_token_id": 50256,
  "classifier_dropout": 0.1,
  "embed_dropout": 0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": null,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neo",
  "num_heads": 12,
  "num_layers": 12

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/merges.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/config.json
Model config GPTNeoConfig {
 

In [27]:
tokenizer.max_model_input_sizes

{'gpt2': 1024,
 'gpt2-medium': 1024,
 'gpt2-large': 1024,
 'gpt2-xl': 1024,
 'distilgpt2': 1024}

In [33]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [28]:
tokenizer.model_max_length = tokenizer.max_model_input_sizes["gpt2"]

In [29]:
def preprocess_function(examples):
    return tokenizer(text=examples["review"],
                      truncation=True,
                      padding="max_length",
                      max_length=tokenizer.max_model_input_sizes["gpt2"])

In [34]:
trainset = trainset.map(preprocess_function,
                        remove_columns=trainset.features.keys(),
                        num_proc=32)
testset = testset.map(preprocess_function,
                      remove_columns=testset.features.keys(),
                      num_proc=32)

Map (num_proc=32):   0%|          | 0/160000 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/20000 [00:00<?, ? examples/s]

# Model + LoRA

In [13]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

In [35]:
model_name_or_path = "EleutherAI/gpt-neo-125M"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    inference_mode=False,
    r=12,
    lora_alpha=32,
    lora_dropout=0.15,
    
    fan_in_fan_out=True,
)

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/config.json
Model config GPTNeoConfig {
  "_name_or_path": "EleutherAI/gpt-neo-125M",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_types": [
    [
      [
        "global",
        "local"
      ],
      6
    ]
  ],
  "bos_token_id": 50256,
  "classifier_dropout": 0.1,
  "embed_dropout": 0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": null,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neo",
  "num_heads": 12,
  "num_layers": 12

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/pytorch_model.bin
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

All model checkpoint weights were used when initializing GPTNeoForCausalLM.

All the weights of GPTNeoForCausalLM were initialized from the model checkpoint at EleutherAI/gpt-neo-125M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPTNeoForCausalLM for predictions without further training.
Generation config file not found, using a generation config created from the model config.
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora.py:230: UserWarning: fan_in_fan_out is set to True but the target module is `torch.nn.Linear`. Setting fan_in_fan_out to False.
  warnings.warn(


trainable params: 442368 || all params: 125640960 || trainable%: 0.3520890002750695


# Training 

In [15]:
from transformers import Trainer, TrainingArguments, TrainerState, TrainerControl, DataCollatorForLanguageModeling
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from transformers.trainer_callback import TrainerCallback
import os

In [16]:
class SavePeftModelCallback(TrainerCallback):
    def on_save(self,
                args: TrainingArguments,
                state: TrainerState,
                control: TrainerControl,
                **kwargs,):
        
        checkpoint_folder = os.path.join(
            args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}"
        )

        kwargs["model"].save_pretrained(checkpoint_folder)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        
        return control

In [17]:
experiment_name = "gpt2-french-allocine-small"
run_name = "experiment-0"

In [36]:
trainer = Trainer(
    model=lora_model, 
    train_dataset=trainset,
    eval_dataset=testset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[SavePeftModelCallback],

    args=TrainingArguments(
        # runtime-related parameters
        auto_find_batch_size = True,
        #gradient_accumulation_steps = 4,
        fp16=True,
        
        # loss minimization-related parameters
        num_train_epochs = 2,
        #warmup_ratio = 0,
        #warmup_steps=100, 
        #max_steps = 200, 
        learning_rate = 2e-5, 
        weight_decay = 0,
        optim = "adamw_torch",
         
        # evaluation-related parameters
        evaluation_strategy = "steps",
        eval_delay = 0,
        eval_steps = 10,
        eval_accumulation_steps = 1,
        
        # log-related parameters
        logging_strategy = "steps",
        logging_first_step = True,
        logging_steps=10, 
        log_level = "info",
        #log_on_each_node = False,
        
        # saving-related parameters
        save_strategy = "steps",
        save_steps = 100,
        save_total_limit = 10,
        
        # path-related parameters
        logging_dir=f"./outputs/runs/{experiment_name}/{run_name}",
        output_dir='outputs',
        overwrite_output_dir = True,
    ),
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [37]:
trainer.train()

***** Running training *****
  Num examples = 160,000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 40,000
  Number of trainable parameters = 442,368
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
***** Running training *****
  Num examples = 160,000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 80,000
  Number of trainable parameters = 442,368


Step,Training Loss,Validation Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1664 in train                    │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/utils/memory.py:124 in decorator              │
│                                                                                                  │
│   121 │   │   │   if batch_size == 0:                                                            │
│   122 │   │   │   │   raise RuntimeError("No executable batch size found, reached zero.")        │
│   123 │   │   │   try:                                                                           │
│ ❱ 124 │   │   │   │   return function(batch_size, *args, **kwargs)                               │
│   125 │   │   │   except Exception as e:                                                         │
│   126 │   │   │   │   if should_reduce_batch_size(e):                                            │
│   127 │   │   │   │   │   gc.collect()                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1945 in _inner_training_loop     │
│                                                                                                  │
│   1942 │   │   │   │   if (                                                                      │
│   1943 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│   1944 │   │   │   │   │   and not is_torch_tpu_available()                                      │
│ ❱ 1945 │   │   │   │   │   and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))          │
│   1946 │   │   │   │   ):                                                                        │
│   1947 │   │   │   │   │   # if loss is nan or inf simply add the average of previous logged lo  │
│   1948 │   │   │   │   │   tr_loss += tr_loss / (1 + self.state.global_step - self._globalstep_  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [38]:
!transformers-cli env

2023-05-25 15:47:29.729324: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
2023-05-25 15:47:36.578636: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.

Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.29.2
- Platform: Linux-5.15.107+-x86_64-with-glibc2.31
- Python version: 3.10.11
- Huggingface_hub version: 0.14.1
- Safetensors version: not installed
- PyTorch version (GPU?): 2.0.1+cu118 (True)
- Tensorflow version (GPU?): 2.12.0 (True)
- Flax version (CPU?/GPU?/TPU?): 0.6.9 (gpu)
- Jax version: 0.4.10
- JaxLib version: 0.4.10
- Using GPU in script?: <fill in>
- Using distributed or parallel set-up in script?: <fill in>

